# Seasonal Migration Patterns of the American White Pelican (Pelecanus erythrorhynchos)

## Species description
The American White Pelican is one of North America's largest birds, measuring from 1.4-1.8 m long with a wingspan of up to 2.4 m (boreal birds website). When I moved to Colorado from the tropics, I was shocked to see such an instantly recognizable and iconic shorebird here along the Front Range. As it turns out, eastern populations of the American White Pelican winter along the Gulf of Mexico and migrate north in the spring to breeding grounds in the northern United States and Canada, passing right through Colorado! My biggest question was "what on earth are they eating, so far from the ocean?" This bird is a non-selective fish eater, foraging from reservoirs and wetlands along its migration route. Instead of diving for fish like the more dramatic seabirds, they will drive fish towards shore in a coordinated flock and scoop up their prey in shallow waters with their enormous bills. 

## Data description
Ecoregion
GBIF Occurrences
... with citations

## Methods description
Normalization - why is it necessary, how did you do it? We normalized by sampling effort in the ecoregion and month, and by area

Add some results (embed map showing species migration over months) - double check the size of the plot, make sure it displays enough. 

## Plot headline
Plot description

## Discussion?

## References

https://www.borealbirds.org/bird/american-white-pelican#:~:text=American%20White%20Pelicans%20are%20gregarious,slowly%20and%20gracefully%20in%20circles.